# Limpieza de Datos (Data Cleaning)

Este notebook está dedicado exclusivamente a la **limpieza y preparación de los datos** del dataset de clientes de telecomunicaciones.
El objetivo es dejar la información lista para el análisis exploratorio (**EDA**).

# 1.-Librerías

In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd #manejo de estructura de datos tabulares

# importamos los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')

# importamos configuracion de pandas
# -----------------------------------------------------------------------
pd.options.display.max_columns = None

# 2.- Cargar datos

In [2]:
# Cargamos los datos desde la carpeta data/raw
df = pd.read_csv("../data/raw/synthetic_personal_finance_dataset.csv")
df.head()

,user_id,age,gender,education_level,employment_status,job_title,monthly_income_usd,monthly_expenses_usd,savings_usd,has_loan,loan_type,loan_amount_usd,loan_term_months,monthly_emi_usd,loan_interest_rate_pct,debt_to_income_ratio,credit_score,savings_to_income_ratio,region,record_date
0,U00001,56,Female,High School,Self-employed,Salesperson,3531.69,1182.59,367655.03,No,NaN,0.00,0,0.00,0.00,0.00,430,8.68,Other,2024-01-09
1,U00002,19,Female,PhD,Employed,Salesperson,3531.73,2367.99,260869.10,Yes,Education,146323.34,36,4953.50,13.33,1.40,543,6.16,North America,2022-02-13
2,U00003,20,Female,Master,Employed,Teacher,2799.49,1003.91,230921.21,No,NaN,0.00,0,0.00,0.00,0.00,754,6.87,Africa,2022-05-12
3,U00004,25,Male,PhD,Employed,Manager,5894.88,4440.12,304815.51,Yes,Business,93242.37,24,4926.57,23.93,0.84,461,4.31,Europe,2023-10-02
4,U00005,53,Female,PhD,Employed,Student,5128.93,4137.61,461509.48,No,NaN,0.00,0,0.00,0.00,0.00,516,7.50,Africa,2021-08-07


# 3.- Transformación de datos

* Mantener user_id como string ( es un identificador alfanumérico).
* Convertir has_loan a booleana (Sí/No --> True/False).
* Pasar columna record_date a datetime para análisis temporal.

## 3.1.- Imputar Nulos

Como ya hemos visto anteriormente, la única columna con nulos es "loan_type" y vimos que los nulos estaban relacionados cuando en la columna "has_loan" había el valor categórico No.
Dicho esto, vamos a imputar los valores nulos con "No_loan".

In [3]:
df["has_loan"].unique()

array(['No', 'Yes'], dtype=object)

In [4]:
df["loan_type"].unique()

array([nan, 'Education', 'Business', 'Car', 'Home'], dtype=object)

A partir de ahora tendremos que hacer cambios en el dataframe, por seguridad vamos a hacer una copia por si más adelante necesitamos comprobar algo o volver atrás.

In [5]:
df_clean = df.copy()
df_clean.head()

,user_id,age,gender,education_level,employment_status,job_title,monthly_income_usd,monthly_expenses_usd,savings_usd,has_loan,loan_type,loan_amount_usd,loan_term_months,monthly_emi_usd,loan_interest_rate_pct,debt_to_income_ratio,credit_score,savings_to_income_ratio,region,record_date
0,U00001,56,Female,High School,Self-employed,Salesperson,3531.69,1182.59,367655.03,No,NaN,0.00,0,0.00,0.00,0.00,430,8.68,Other,2024-01-09
1,U00002,19,Female,PhD,Employed,Salesperson,3531.73,2367.99,260869.10,Yes,Education,146323.34,36,4953.50,13.33,1.40,543,6.16,North America,2022-02-13
2,U00003,20,Female,Master,Employed,Teacher,2799.49,1003.91,230921.21,No,NaN,0.00,0,0.00,0.00,0.00,754,6.87,Africa,2022-05-12
3,U00004,25,Male,PhD,Employed,Manager,5894.88,4440.12,304815.51,Yes,Business,93242.37,24,4926.57,23.93,0.84,461,4.31,Europe,2023-10-02
4,U00005,53,Female,PhD,Employed,Student,5128.93,4137.61,461509.48,No,NaN,0.00,0,0.00,0.00,0.00,516,7.50,Africa,2021-08-07


In [6]:
#Imputaremos los nulos No_loan donde realmente no hay préstamo
mask_no_loan = df_clean['has_loan'].eq('No')
df_clean.loc[mask_no_loan, 'loan_type'] = df_clean.loc[mask_no_loan, 'loan_type'].fillna('No_loan')

In [9]:
df_clean["loan_type"].isnull().sum()

np.int64(0)

No hay ningún nulo, por lo tanto, todos los nulos correspondían a que el usuairo no tenía préstamos.

In [7]:
df_clean["loan_type"]

0          No_loan
1        Education
2          No_loan
3         Business
4          No_loan
           ...    
32419          Car
32420         Home
32421      No_loan
32422      No_loan
32423      No_loan
Name: loan_type, Length: 32424, dtype: object

Vemos que los nulos ya han sido imputados correctamente.

In [10]:
# Reseteamos los ínidices
df_clean.reset_index(drop=True,inplace=True)

## 3.1.- Formateo de nombres de columnas

Antes de cambiar el tipo de datos vamos a formatear el nombre de las columnas por si hubiesen espacios o caracteres especiales.

In [11]:
df_clean.columns

Index(['user_id', 'age', 'gender', 'education_level', 'employment_status',
       'job_title', 'monthly_income_usd', 'monthly_expenses_usd',
       'savings_usd', 'has_loan', 'loan_type', 'loan_amount_usd',
       'loan_term_months', 'monthly_emi_usd', 'loan_interest_rate_pct',
       'debt_to_income_ratio', 'credit_score', 'savings_to_income_ratio',
       'region', 'record_date'],
      dtype='object')

In [12]:
# todas las letras en minusculas
df_clean.columns = df.columns.str.lower()

#editamos el nombre de las columnas 
df_clean.columns = df.columns.str.replace(" ", "_")

df_clean.columns

Index(['user_id', 'age', 'gender', 'education_level', 'employment_status',
       'job_title', 'monthly_income_usd', 'monthly_expenses_usd',
       'savings_usd', 'has_loan', 'loan_type', 'loan_amount_usd',
       'loan_term_months', 'monthly_emi_usd', 'loan_interest_rate_pct',
       'debt_to_income_ratio', 'credit_score', 'savings_to_income_ratio',
       'region', 'record_date'],
      dtype='object')

## 3.3.- Cambiar Dtype columnas

Cambiamos el tipo de dato de las columnas `Count` , `Tenure Months` `Churn Value`, ` monthly_charges`y `Churn Score` a `int64` y `Zip Code` a `str`.

In [13]:
# Identificar columnas categóricas y numéricas
columnas_categoricas = df_clean.select_dtypes(include=['object']).columns.tolist()
columnas_numericas = df_clean.select_dtypes(include=['int', 'float']).columns.tolist()
print(columnas_categoricas)
print(columnas_numericas)

['user_id', 'gender', 'education_level', 'employment_status', 'job_title', 'has_loan', 'loan_type', 'region', 'record_date']
['age', 'monthly_income_usd', 'monthly_expenses_usd', 'savings_usd', 'loan_amount_usd', 'loan_term_months', 'monthly_emi_usd', 'loan_interest_rate_pct', 'debt_to_income_ratio', 'credit_score', 'savings_to_income_ratio']
